# SI 370 - Homework 1 - Python, Pandas, and Synthetic Data
## Due: Friday, September 22, 2017.  11:59pm

### Submit this file via Canvas Assignments.

The purpose of this lab is to give you a chance to put into practice some of the things you know about python and pandas.  

We are going to extend the data from the class survey by accessing a large, synthetic data set (from the Practical Learning Analytic MOOC offered last year).  Basically, you're going to access historical data on student performance in a course and use that, along with survey data, to guide the delivery of tailored messages to a student.

You should download the following data files from teh Datasets folder in the Canvas Files: ```student.record.csv```, ```student.course.csv```, ```term.table.txt```, and ```deidentified-370-survey-anonid.csv```.  Note that the last file is different than the one we used in this week's lab.

A quick note about the data:  whereas the deidentified survey results are from your responses in the first class, the synthetic data are randomly chosen from the larger data set.  Because the synthetic data set is completely deidentified (and, indeed, does not contain any values that exist in the real data set -- we'll talk about that in class), it's impossible for me to derive who maps onto the synthetic data set.  So we're just going to pick some data from the synthetic data set to complete this assignment.

# Please provide:

## Your name: [Alexander "AJ" Goldstein] 

## Your uniqname: [ajva] 

## People you worked with: [None]

First, let's do the usual importing of pandas and numpy.  We'll also issue a directive to keep pandas from complaining later on.

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
# This next line just suppresses an unnecessary warning message later on
pd.options.mode.chained_assignment = None  # default='warn'

We are going to read in two data files, both of which are descibed on the [PLA Wiki](https://github.com/bkoester/PLA/wiki#data).  ```student.record.csv``` holds details about each student, and ```student.course.csv``` contains records that correspond to each course taken by every student over the few decades.  We will be focusing mostly on ```student.course.csv``` but the bonus question requires the use of ```student.record.csv``` and ```terms.table.txt``` so we'll load them here too.

In [3]:
studentRecord = pd.read_csv('student.record.csv')
studentCourse = pd.read_csv('student.course.csv')
terms = pd.read_csv('term.table.txt',sep='\t')

Now let's take a look at the first few lines of both of those tables.  To understand what the different columns are read the [PLA Wiki](https://github.com/bkoester/PLA/wiki#data).

In [4]:
studentCourse.head()

ANONID SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  ANON_INSTR_ID  \
0      26     ACC          272               2.0  3.343636   P           2920   
1     114     ACC          272               2.0  2.817857   P           2920   
2     121     ACC          272               4.0  4.000000   P            201   
3     125     ACC          272               1.3  3.639063   P           3360   
4     180     ACC          272               3.0  3.846154   P           2920   

   TERM  
0    79  
1    83  
2   111  
3    84  
4   107

Let's take a look at the columns, just to be sure they match the documentation.

In [10]:
studentRecord.columns

Index(['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA',
       'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1',
       'STDNT_GROUP2', 'MAJOR1_DEPT', 'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID',
       'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM'],
      dtype='object')

Now filter the studentCourse DataFrame to only include those rows that correspond to STATS 250 students who received standing in the course (i.e. the grade points, or GRD_PTS_PER_UNIT > 0.0).  Note that 'STATS' is the 'SUBJECT' and '250' is the 'CATALOG_NBR'.

In [14]:
studentCourse250 = studentCourse[(studentCourse['GRD_PTS_PER_UNIT'] > 0.0) & (studentCourse['SUBJECT'] == 'STATS') & (studentCourse['CATALOG_NBR'] == 250)]

ANONID SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  \
877249       8   STATS          250               3.7  3.502564   S   
877250      48   STATS          250               4.0  3.936842   S   
877251      67   STATS          250               4.0  3.809091   S   
877252      99   STATS          250               2.3  3.398684   S   
877253     124   STATS          250               4.0  3.877778   S   

        ANON_INSTR_ID  TERM  
877249           1260   130  
877250           1260   119  
877251           1260   135  
877252            386   122  
877253           1260   131

And take a peek at the results

In [15]:
studentCourse250.head()

ANONID SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  \
877249       8   STATS          250               3.7  3.502564   S   
877250      48   STATS          250               4.0  3.936842   S   
877251      67   STATS          250               4.0  3.809091   S   
877252      99   STATS          250               2.3  3.398684   S   
877253     124   STATS          250               4.0  3.877778   S   

        ANON_INSTR_ID  TERM  
877249           1260   130  
877250           1260   119  
877251           1260   135  
877252            386   122  
877253           1260   131

Let's get some basic statistics about the different columns:

In [16]:
studentCourse250.describe()

ANONID  CATALOG_NBR  GRD_PTS_PER_UNIT          GPAO  \
count   15781.000000      15781.0      15781.000000  15781.000000   
mean   106280.032698        250.0          3.189773      3.347299   
std     61498.896860          0.0          0.770321      0.436231   
min         8.000000        250.0          0.700000      1.300000   
25%     53361.000000        250.0          2.700000      3.092308   
50%    106262.000000        250.0          3.300000      3.404225   
75%    159484.000000        250.0          3.700000      3.666667   
max    214162.000000        250.0          4.400000      8.100000   

       ANON_INSTR_ID          TERM  
count   15781.000000  15781.000000  
mean      893.487548    127.179330  
std       539.049188      5.743606  
min        71.000000    118.000000  
25%       386.000000    122.000000  
50%      1260.000000    127.000000  
75%      1260.000000    131.000000  
max      1802.000000    135.000000

That's fine for an overview, but we'll need to extract the mean value for GRD_PTS_PER_UNIT by itself because in the next step we're going to want to create a new column that represents whether the value of GRD_PTS_PER_UNIT is greater or less than the mean.  So let's try that out.

In [18]:
meanGrade = studentCourse250['GRD_PTS_PER_UNIT'].mean()
meanGrade

3.1897725112476403

Now that we know how to get that number, use the same function to create a new column called "overunder" in the studentCourse250 data frame that represents the differene between the GRD_PTS_PER_UNIT and the mean GRD_PTS_PER_UNIT.  Do not "hard code" the actual mean value... that is, do not type "3.1897725112476403" in the next cell.

In [28]:
studentCourse250["overunder"] = studentCourse250['GRD_PTS_PER_UNIT'] - meanGrade

While we're in there, let's calculate some values of interest.  For example, what proportion of students get a 4.0 in STATS 250?

In [26]:
# insert a single line of code to return the number of rows that have 
# GRD_PTS_PER_UNIT equal to 4.0
len(studentCourse250[studentCourse250['GRD_PTS_PER_UNIT'] == 4.0])

3601

The next thing we're going to do is merge the survey results with the results from the PLA data set.  To set ourselves up to do this we're going to need to index both data frames on the same column.  Hint:  use the set_index(...) function on both the studentCourse250 and survey data frames. Start with the studentCourse250 data frame.

In [27]:
studentCourse250.set_index('ANONID')

SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  ANON_INSTR_ID  \
ANONID                                                                       
8        STATS          250               3.7  3.502564   S           1260   
48       STATS          250               4.0  3.936842   S           1260   
67       STATS          250               4.0  3.809091   S           1260   
99       STATS          250               2.3  3.398684   S            386   
124      STATS          250               4.0  3.877778   S           1260   
135      STATS          250               2.3  2.780952   S            386   
136      STATS          250               1.0  2.812069   S            386   
144      STATS          250               4.0  3.953846   S           1802   
161      STATS          250               4.0  4.000000   S           1260   
169      STATS          250               3.7  3.200000   S           1260   
175      STATS          250               4.0  3.268254   S           1260   
212      STATS          250               2.3  2.773333   S            386   
219      STATS          250               4.0  3.528571   S           1260   
231      STATS          250               2.3  3.055556   S           1260   
249      STATS          250               3.3  3.334783   S           1260   
255      STATS          250               3.3  3.472727   S           1446   
270      STATS          250               2.3  3.284615   S            386   
290      STATS          250               3.7  3.666667   S           1260   
311      STATS          250               3.7  3.640909   S            386   
359      STATS          250               4.0  3.457627   S           1260   
364      STATS          250               3.7  3.490244   S             71   
388      STATS          250               4.0  3.759091   S           1260   
390      STATS          250               2.0  2.648000   S            386   
392      STATS          250               4.0  3.875862   S           1802   
398      STATS          250               4.0  3.671429   S            386   
402      STATS          250               3.7  3.651852   S            386   
407      STATS          250               3.0  3.358333   S            386   
431      STATS          250               3.7  3.728000   S             71   
434      STATS          250               4.0  3.912000   S            386   
437      STATS          250               3.3  3.095238   S            386   
...        ...          ...               ...       ...  ..            ...   
213699   STATS          250               3.0  3.147170   S           1260   
213707   STATS          250               3.7  3.900000   S           1260   
213712   STATS          250               3.7  3.676596   S           1260   
213716   STATS          250               3.3  3.212000   S            386   
213718   STATS          250               3.7  3.918182   S            386   
213723   STATS          250               3.7  3.326829   S           1260   
213727   STATS          250               4.0  4.000000   S           1260   
213728   STATS          250               3.7  3.398750   S            386   
213759   STATS          250               3.7  3.346341   S           1260   
213811   STATS          250               4.0  3.877778   S           1446   
213824   STATS          250               3.0  2.945614   S            386   
213847   STATS          250               2.7  3.078313   S           1260   
213853   STATS          250               3.7  3.828571   S            386   
213854   STATS          250               2.3  2.257407   S             71   
213861   STATS          250               3.7  3.424444   S           1260   
213871   STATS          250               4.0  3.395349   S           1802   
213877   STATS          250               3.7  3.450877   S            386   
213878   STATS          250               3.7  3.782143   S           1802   
213907   STATS          250  

Now load the survey data into another data frame.  Note that we're using a slightly different csv file, one with ANONID values.  This will allow us to merge the survey data with the PLA data frame.

In [31]:
survey=pd.read_csv('deidentified-370-survey-anonid.csv')

In [32]:
survey.head()

First name Last name What grade do you expect to get in this course?  \
0  September   Maestas                                               A   
1      Jarod     Tracy                                               A   
2      Digna    Dickey                                              A-   
3      Minda   Mcclain                                               A   
4        Lan     Pitts                                               A   

  How many hours outside of lecture and lab do you plan to spend on this course (per week)?  \
0                                           12 hours                                          
1                                                 10                                          
2                                 As much as needed?                                          
3                             10-15 (hopefully less)                                          
4                                                  6                                          

   How enthusiastic are you about taking this course?  \
0                                                  3    
1                                                  5    
2                                                  4    
3                                                  4    
4                                                  5    

   How worried are you about this course?  \
0                                       4   
1                                       2   
2                                       4   
3                                       4   
4                                       3   

  In Python, how do you create a variable "a" that is equal to 2?  \
0                                                a=2                
1                                                a=2                
2                                                a=2                
3                                                a=2                
4                                                a=2                

  How would you cast the string variable "a" that is equal to "2" into the integer 2?  \
0                                             int(a)                                    
1                                             int(a)                                    
2                                             int(a)                                    
3                                             int(a)                                    
4                                             int(a)                                    

  Which of the following is a valid dictionary in Python?  \
0       myExample = {'someItem': 2, 'otherItem': 20}        
1       myExample = {'someItem': 2, 'otherItem': 20}        
2       myExample = {'someItem': 2, 'otherItem': 20}        
3       myExample = {'someItem': 2, 'otherItem': 20}        
4       myExample = {'someItem': 2, 'otherItem': 20}        

  What will you see if you execute the following:  \
0                                          [4, 5]   
1                                          [4, 5]   
2                                          [4, 5]   
3                                          [4, 5]   
4                                          [4, 5]   

   What will the output of the following code be?  \
0                                             5.0   
1                                             5.0   
2                                             5.0   
3                                             1.0   
4                                             5.0   

   What grade did you give yourself?  ANONID  
0                                  8  116486  
1                                  9   12498  
2                                  7  115540  
3                                  6  167810  
4                                 10  141957

Let's shorten the column names so they're more manageable.

In [33]:
survey.columns=['first','last','grade','hours','enth','worry','q1','q2','q3','q4','q5','overall','ANONID']

And let's take a look to see what we have.

In [34]:
survey.head()

first     last grade                   hours  enth  worry   q1      q2  \
0  September  Maestas     A                12 hours     3      4  a=2  int(a)   
1      Jarod    Tracy     A                      10     5      2  a=2  int(a)   
2      Digna   Dickey    A-      As much as needed?     4      4  a=2  int(a)   
3      Minda  Mcclain     A  10-15 (hopefully less)     4      4  a=2  int(a)   
4        Lan    Pitts     A                       6     5      3  a=2  int(a)   

                                             q3      q4   q5  overall  ANONID  
0  myExample = {'someItem': 2, 'otherItem': 20}  [4, 5]  5.0        8  116486  
1  myExample = {'someItem': 2, 'otherItem': 20}  [4, 5]  5.0        9   12498  
2  myExample = {'someItem': 2, 'otherItem': 20}  [4, 5]  5.0        7  115540  
3  myExample = {'someItem': 2, 'otherItem': 20}  [4, 5]  1.0        6  167810  
4  myExample = {'someItem': 2, 'otherItem': 20}  [4, 5]  5.0       10  141957

And let's set the index column to be the same as what we set the studentCourse250 index to.

In [35]:
survey.set_index('ANONID')

first         last grade  \
ANONID                                 
116486  September      Maestas     A   
12498       Jarod        Tracy     A   
115540      Digna       Dickey    A-   
167810      Minda      Mcclain     A   
141957        Lan        Pitts     A   
111827     Rosina      Venegas     A   
72046     Candyce       Thorpe     A   
165604      Garth        Fitch     A   
4571      Mafalda      Bullock     A   
108920      Twana       Larose    A-   
180420      Loyce       Hewitt     A   
1675     Barabara         Lott     A   
172054     Genaro         Helm     A   
194197     Karisa         Cave     A   
161053     Hertha  Crutchfield    A-   
45037      Pattie       Curley    A-   
86220     Harmony       Madden    A-   
82202       Ivana         Leak     A   
199435   Latarsha      Snowden    A-   
129138    Imogene      Provost     A   
124278  Nicolette      Rickard     A   
80559          Pa       Mcleod     A   
53372        Amal         Vann    A-   
58998      Gennie         Frey     A   
57619      Collen     Stauffer     A   
172679     Janeen       Brower     A   
201248     Oneida      Soriano     A   
105784      Adell       Bustos     B   
91605      Holley     Olmstead     A   
17012      Makeda       Corona     A   
188041     Lawana  Mccallister     A   
190830     Iliana        Milam     A   
5713       Leisha        Craft    A-   
62761      Susana       Saxton    A+   
34268    Sherrill       Bowens     A   
95956          Ha     Cranford     A   
129386   Angelita      Stephen    A-   
171780     Travis        Dwyer    A+   
139816     Lemuel   Stephenson     A   
144504       Bong  Weatherford    A-   
183186   Latashia         Mead    A-   
163125   Laurette  Christenson     A   
97687     Devorah      Sherrod     A   

                                                    hours  enth  worry  \
ANONID                                                                   
116486                                           12 hours     3      4   
12498                                                  10     5      2   
115540                                 As much as needed?     4      4   
167810                             10-15 (hopefully less)     4      4   
141957                                                  6     5      3   
111827                                              6-May     3      3   
72046                                          at least 6     4      3   
165604                                                  6     4      1   
4571                                                   10     4      2   
108920                                                  5     3      5   
180420                                                  4     4      1   
1675                                                6-May     4      4   
172054                                                  8     3      4   
194197                                                 10     5      2   
161053                                                 10     3      3   
45037                                              12-Sep     4      3   
86220                                                  10     3      4   
82202                                                  10     4      4   
199435                                                 10     3      5   
129138                                                  4     4      1   
124278                                                 10     4      1   
80559                                                   4     4      3   
53372                                                   6     4      4   
58998                                                  10     5      5   
57619   A lot. I have a feeling this will be my most c...     4      4   
172679                                                  4     2      4   
201248                                                 10     4      4   
105784                                                  4     3  

Now for the magic (well, not really if you remember how SQL works).

We're going to join the two data frames.  Read the documentation for pd.merge and try to do the appropriate join of the two data frames.  If you're successful, you should see data from both the survey data and the PLA data in each line.



In [41]:
joined = pd.merge(studentCourse250, survey, how='right')

And look at the first few lines to make sure it worked.

In [42]:
joined.head()

ANONID SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  ANON_INSTR_ID  \
0    1675   STATS          250               2.0  2.563793   S             71   
1    4571   STATS          250               3.0  3.279167   S            386   
2    5713   STATS          250               3.0  3.323077   S           1260   
3   12498   STATS          250               3.0  3.501639   S            386   
4   17012   STATS          250               3.7  3.710870   S           1260   

   TERM  overunder     first   ...    grade  hours enth  worry       q1  \
0   130  -1.189773  Barabara   ...        A  6-May    4      4      a=2   
1   134  -0.189773   Mafalda   ...        A     10    4      2      a=2   
2   122  -0.189773    Leisha   ...       A-      7    4      4  int a=2   
3   135  -0.189773     Jarod   ...        A     10    5      2      a=2   
4   118   0.510227    Makeda   ...        A     10    5      5      a=2   

       q2                                            q3          q4   q5  \
0  int(a)  myExample = {'someItem': 2, 'otherItem': 20}      [4, 5]  5.0   
1  int(a)  myExample = {'someItem': 2, 'otherItem': 20}      [4, 5]  5.0   
2  int(a)  myExample = {'someItem': 2, 'otherItem': 20}  4, 5, 2006  5.0   
3  int(a)  myExample = {'someItem': 2, 'otherItem': 20}      [4, 5]  5.0   
4  int(a)  myExample = {'someItem': 2, 'otherItem': 20}      [4, 5]  5.0   

   overall  
0       10  
1       10  
2        8  
3        9  
4       10  

[5 rows x 21 columns]

Create a column for numeric GPAs that correspond to the expected grades based on the following table:

|Letter|GPA|
|:------|-----------|
|A+|4.0||
|A|4.0|
|A-|3.7|
|B+|3.3|
|B|3.0|
|B-|2.7|
|C+|2.3|
|C|2.0|
|C-|1.7|
|D+|1.3|
|D|1.0|
|D-|0.7|
|E|0.0|

In [44]:
# Here's a hint about one way to do it.  If you use this please
# review the 'map' and 'lambda' commands
gpaMap = {'A+': 4.0, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0, 'C-': 1.7, 'D+': 1.3, 'D': 1.0, 'D-': 0.7, 'E': 0.0}
joined['gpa'] = joined['grade'].map(lambda letterGrade: gpaMap[letterGrade])

Now calculate the mean expected GPA for the course and create a new
 column that represents the difference between a student's expected GPA
 and the mean GPA

In [45]:
joined['gpaOverUnder'] = joined['gpa'] - joined['gpa'].mean()

And now do the same with worry: create a column that represents the difference between each individual student's worry and the overall mean worry.

In [129]:
joined['enthOverUnder'] = joined['enth'] - joined['enth'].mean()
joined['worryOverUnder'] = joined['worry'] - joined['worry'].mean()
joined.head()
joined[joined.ANONID == int("116486")]

ANONID SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  \
22  116486   STATS          250               3.7  3.525862   S   

    ANON_INSTR_ID  TERM  overunder      first      ...         q1      q2  \
22             71   131   0.510227  September      ...        a=2  int(a)   

                                              q3      q4   q5 overall  gpa  \
22  myExample = {'someItem': 2, 'otherItem': 20}  [4, 5]  5.0       8  4.0   

   gpaOverUnder worryOverUnder  enthOverUnder  
22     0.097727       0.681818      -0.840909  

[1 rows x 25 columns]

You now have three axes that represent the difference between three variables (the student's performance in STATS 250 compared to others who have taken that course, the student's enthusiasm relative to other students in the course, and the student's worry relative to other students in the course).

Let's use the following representations: 
```
S = Above average in STATS250, s = below average in STATS 250
G = Estimated grade in 370 to be above mean; g = estimated grade in 370 to be below mean
E = Above average Enthusiasm, e = below average enthusiasm
W = Above average Worry, w = below average worry
```
So each student can be represented as a combination of those four dimensions: SgeW, sGeW, SGew, etc. for a total of 16 different possible combinations. 



Write code that prints out the number of students that fall into each of the 16 categories.

In [51]:
# Replace this cell with code that prints out the following (with the correct values)

SGEW_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] > 0)])
sgew_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] < 0)])

SGEw_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] > 0)])
SGeW_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] < 0)])
SgEW_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] > 0)])
sGEW_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] > 0)])

SGew_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] < 0)])
SgeW_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] < 0)])
sGEw_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] > 0)])
sGeW_count = len(joined[(joined['overunder'] <0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] < 0)])
sgEW_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] > 0)])
SgEw_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] > 0)])

Sgew_count = len(joined[(joined['overunder'] > 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] < 0)])
sGew_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] > 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] < 0)])
sgeW_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] > 0) & (joined['enthOverUnder'] < 0)])
sgEw_count = len(joined[(joined['overunder'] < 0) & (joined['gpaOverUnder'] < 0) & (joined['worryOverUnder'] < 0) & (joined['enthOverUnder'] > 0)])

Think about a suitable message that you would like to send students in each of those conditions and put them into an appropriate data structure (a dictionary might be a good choice.

In [109]:
# create an appropriate data structure here
# something like
messages = {}

# two extremes
messages[('S','G','E','W')] = """
You did great in STATS 250, and you've set the bar high for yourself in this course.  
Your enthusiasm should carry you a long way, but try to be less anxious about the course!
"""
messages[('s','g','e','w')] = """
Based off your grades and enthusiasm, it looks like you're screwed! But at least you're not worried about it :)
"""

## one lowercase
messages[('S','G','E','w')] = """
Looks like you did great in STATS 250 and you're planning to do well here too.
You've got a good combination of relaxation and enthusiasm going got you... try to spread
that approach to your classmates.
"""

messages[('S','G','e','W')] = """
Looks like you did great in STATS 250 and you're planning to do well here too.
But you seem a little worried about the course -- try coming to office hours if you struggle.
"""

messages[('S','g','E','W')] = """
Looks like you did great in STATS 250 but you're projected to do poorly here.
Based off your worry score, you seem pretty aware of how bad things could get! :P Stay on top of things.
"""

messages[('s','G','E','W')] = """
Looks like you did bad in STATS 250 and but you're planning to do well here.
No need to be worried; stay enthusiastic and all will be gucci.
"""

## two each
messages[('S','G','e','w')] = """
Looks like you did great in STATS 250 and you're planning to do well here too.
Your worry score is low, but you seem a little TOO CHILL about this class. Try taking a 5 hour energy.
"""

messages[('S','g','e','W')] = """
Looks like you did great in STATS 250 but you're projected to do poorly here.
Your worry score is high, justifiably so, but you gotta boost that enthusiasm if you're going to have a fighting chance!
"""

messages[('s','G','E','w')] = """
Looks like you did bad in STATS 250 but you're planning to do well here.
You seem pretty enthusiastic and relaxed about your chances to turn your luck around. Good luck with that!
"""

messages[('s','G','e','W')] = """
Looks like you did bad in STATS 250 but you're planning to do well here.
No need to be worried! Just boost that enthusiasm a bit and you'll be on your way to a better life!
"""

messages[('s','g','E','W')] = """
Looks like you did bad in STATS 250 and you're equally screwed here.
No need for that enthusiasm -- you really should be very worried.
"""

messages[('S','g','E','w')] = """
Looks like you did great in STATS 250 but your luck is about to end.
No need for that enthusiasm; you probably should be worrying more. Looks like your good fortunate is over :P
"""

## one caps
messages[('S','g','e','w')] = """
Looks like you did great in STATS 250 but your luck is about to end.
Based off your worry and enthusiasm scores, it seems you've got a bad case of senioritis.
"""

messages[('s','G','e','w')] = """
Looks like you did bad in STATS 250 but you're planning to do well here.
Boost that enthusiasm a bit; you're luck is about to turn around!
"""

messages[('s','g','E','w')] = """
Looks like you did bad in STATS 250 and you're equally screwed here.
Your worry score is low, but you really should be terrified. Wipe that enthusiasm off your face and start working!
"""

messages[('s','g','e','W')] = """
Looks like you did bad in STATS 250 and you're projected to do just as terrible here.
Based off your worry score, I'm glad you're at least aware of how screwed you are :)
"""

Finally, create a function that returns the appropriate message for a student given
either the student's first name, last name, or ANONID.  As usual, the function should return an appropriate message if no student matches the criteria.

In [140]:
def printMessage(identifier):
    if (identifier[0].isdigit()):
        record = joined[(joined['ANONID'] == int(identifier))]
    
    else:
        record = joined[(joined['first'] == identifier) | (joined['last'] == identifier)]
    
    if (record.empty):
        print("the student you're searching for was not found.")
        
    else:
        sStatus = 'S'
        if (record['overunder'] < 0).values:
            sStatus = 's'
        gStatus = 'G'
        if (record['gpaOverUnder'] < 0).values: gStatus = 'g'
        eStatus = 'E'
        if (record['enthOverUnder'] < 0).values: eStatus = 'e'
        wStatus = 'W'
        if (record['enthOverUnder'] < 0).values: wStatus = 'w'
    
        print(messages[sStatus,gStatus,eStatus,wStatus])

In [142]:
# The following should all return an appropriate message
printMessage("Karisa")
printMessage("Madden")
printMessage("116486")
# The following should return a message indicating the student was not found
printMessage("Chris")



You did great in STATS 250, and you've set the bar high for yourself in this course.  
Your enthusiasm should carry you a long way, but try to be less anxious about the course!


Looks like you did great in STATS 250 but your luck is about to end.
Based off your worry and enthusiasm scores, it seems you've got a bad case of senioritis.


Looks like you did great in STATS 250 and you're planning to do well here too.
Your worry score is low, but you seem a little TOO CHILL about this class. Try taking a 5 hour energy.

the student you're searching for was not found.


## Bonus: 
In your message, include information about their standing (Freshman, Sophomore, Junior, or Senior) when they took STATS 250 based on the difference between the TERM value in the student.course.csv file and the ADMIT_TERM value in student.record.csv.  You'll also need to access the data frame from the term.table.txt file to do this.